# day02 - И волки сыты

---

In [1]:
from random import randint, betavariate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta, date, time
import pylab
import seaborn as sns
plt.style.use('seaborn')
%matplotlib inline
pylab.rcParams['figure.figsize'] = (30, 20)

---

## Моменты времени и протяженность во времени

Когда произошло событие? Сколько времени оно длилось? Эти два вопроса отличают моменты от продолжительности.  
Если уместно задать первый вопрос, или его аналог, то речь идет о моменте времени. Уместность второго вопроса свидетельствует о продолжительности. Данные по моментам и продолжительностям имеют разные сущности и требуют различные инструменты обработки при анализе. Рассмотрим на примере:

---

В файле *iCafe_train.xlsx* представлены данные одного дня интернет-кафе. Кафе принимает заказы на готовую еду и доставляет ее клиентам.  
Таблица содержит столбцы:
- Order_time - содержит информацию о времени размещения заказа на сайте, время представлено строкой вида 'h:mm:ss';
- Delivery_time - содержит информацию о времени, затраченном на доставку, время указано целым числом и характеризует количество минут, потребовавшихся на доставку от помещения кафе до клиента;
- Office - характеристика места доставки:
    - 1 - если доставка осуществлялась к месту работы клиента;
    - 0 - если доставка осуществлялась в жилой сектор. 

Нашей задачей будет определить, куда доставлялся заказ, на основании данных о времени его размещения и доставки.

Для решения задачи нам потребуется несколько библиотек. Часть из них уже импортированы в первой ячейке ноутбука, вторая часть появится по ходу решения задачи. Среди импортированных уже известные нам библиотеки numpy и matplotlib, а так же новые библиотеки:
- ***pandas*** - библиотека инструментов для работы с одномерными, двумерными и трехмерными структурами данных. Чаще всего работают с двумерными структурами данных (таблицами), которые называются датафреймами (DataFrame). Информация по pandas:
    - [Официальная документация](https://pandas.pydata.org)
    - [Краткий справочник на русском языке](https://pythonworld.ru/obrabotka-dannyx/pandas-cookbook-0-ipython.html)
- ***seaborn*** - библиотека инструментов визуализаци статистических данных. 
[Официальная документация](https://seaborn.pydata.org)

Для начала импортируем данные из файла:

In [ ]:
from google.colab import drive  # если вы выполняете код из среды Google Colab, нужно подключить свой гугл-диск,
drive.mount('/content/drive')   # чтобы можно было оттуда считать файл с данными для этого задания

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/data/iCafe_train.xlsx') # Создаем датафрейм df из файла
df                                                                                    # Смотрим, что в нем располагается

Строковые значение столбца Order_time не удобны с точки зрения математического анализа и визуализации, поэтому необходимо преобразовать данные к другому виду. Создадим столбец Order_time_sec, в котором будет представлена информация о времени размещения заказа в секундах. Для этого воспользуемся двумя функциями:
- ***to_timedelta*** модуля pandas, которая распарсит строку времени и определит отклонение результата от 0;
- ***timedelta64*** модуля numpy, которая позволит нам получить в знаменателе 1 секунду и, соответственно, весь результат в секундах. Изменение параметра 's' на 'm' или на 'h' позволит получать результат в минутах и часах соответственно.

In [ ]:
df['Order_time_sec'] = pd.to_timedelta(df.Order_time) // np.timedelta64(1,'s') # Создаем столбец
df                                                                             # Смотрим результат

In [ ]:
df.describe()

Начинать анализ следует с того, чтобы понять что вообще происходит, какие наблюдаются правила, зависимости, попытаться понять процесс...

Для этого лучше всего визуализировать данные. Далее мы построим несколько графиков и попытаемся их интерпретировать.

In [ ]:
"""Строим график, где по оси абсцисс откладывается время поступления заказа, а по оси ординат место доставки"""

plt.scatter(df.Order_time_sec, df.Office)
plt.xlabel('Время поступления заказа (Order_time_sec)')
plt.ylabel('Место доставки (Office)')
plt.show()

Судя по графику, заказы в офис и не в офис по моменту поступления пересекаются, но кажется, что не очень сильно. Точнее мы сейчас определить не сможем.

In [ ]:
"""Строим график, где по оси абсцисс откладывается время доставки заказа, а по оси ординат место доставки"""

plt.scatter(df.Delivery_time, df.Office)
plt.xlabel('Время доставки заказа (Delivery_time)')
plt.ylabel('Место доставки (Office)')
plt.show()

Получили график, который не дает нам никакой информации. Это получилось в следствие того, что время доставки гораздо более дискретная величина, по сравнению с моментом размещения заказа, и у нас есть заказы из обоих классов почти для всех минут доставки. Построим другой график:

In [ ]:
"""Строим диаграмму, где на каждое время доставки представлено количество доставок в офис и не в офис"""

sns.countplot(x = 'Delivery_time', hue = 'Office', data = df)
plt.show()

Из диаграммы мы видим, что в среднем в офис поставка осуществляется быстрее. Скорее всего помещение кафе располагается ближе к офисам, чем к жилым кварталам. Также мы видим, что чем дольше время доставки, тем меньше кафе получает заказов.

Попробуем построить подобный график для момента размещения заказа:

In [ ]:
"""Строим диаграмму, где на каждое время размещения заказа представлено количество доставок в офис и не в офис"""

sns.countplot(x = 'Order_time_sec', hue = 'Office', data = df)
plt.show()

Опять получили лишенный смысла график. Здесь наблюдается обратная проблема. Заказы не поступают одноременно (за исключением одного случая, который следует рассматривать как случайное и крайне редкое событие). Для того, чтобы получить информативный график, нам необходимо собрать моменты поступления заказов во временные диапазоны, например, часовые. Создадим столбец Order_time_h, в который запишем час поступления заказа и построим диаграмму на полученных данных:

In [ ]:
"""Строим диаграмму, где на каждый час времени размещения заказа представлено количество доставок 
   в офис и не в офис"""

df['Order_time_h'] = df.Order_time_sec//3600
sns.countplot(x = 'Order_time_h', hue = 'Office', data = df)
plt.show()

Вот теперь мы видим, что по моменту размещения заказа доля пересечения классов действительно не велика. Так же мы видим, что есть три больших всплеска заказов: 
- с 11 до 14 часов в офис - обеденный перерыв;
- с 18 до 22 часов на дом - для тех кто не хочет или не может готовить ужин;
- с 23 до 1 часа ночи в офис - дежурные на непрерывном производстве (например, тех. поддержка провайдеров)

Построим еще пару графиков. Постарайтесь их интерпретировать самостоятельно.

In [ ]:
plt.scatter(df.Order_time_sec[df.Office == 1], df.Delivery_time[df.Office == 1], color = 'red')
plt.scatter(df.Order_time_sec[df.Office == 0], df.Delivery_time[df.Office == 0])
plt.legend(['В офис', 'На дом'])
plt.xlabel('Время поступления заказа (Order_time_sec)')
plt.ylabel('Время доставки заказа (Delivery_time)')
plt.show()

In [ ]:
sns.pairplot(df[['Office', 'Order_time_sec', 'Delivery_time']], hue="Office", height=6)
plt.show()

---

### Разработка классификатора
***Логистическая регрессия***  
Настало время создать классификатор, который будет на основании данных о моменте размещения заказа и времени его доставки определять класс заказа (в офис, не в офис). Классификатор будем строить с помощью логистической регрессии:  
$$f(z)=\frac{1}{1+e^{-z}}$$
где *z* - это уравнение линейной регрессии по *n* признакам:
$$z=\theta_0+\sum_{i=1}^n {\theta_ix_i}$$
Поиск вектора $\bar{\theta}$ и является задачей классификации с помощью [логистической регрессии](https://habr.com/ru/company/io/blog/265007/).

***Метрики качества***  
Разработанный классификатор предсказывает к какому классу принадлежит объект на основании значений его признаков. В результате возможны четыре ситуации:
- Объект имеет характеристику класса 1 и классификатор определяет характеристику как 1. Это означает, что классификатор сработал верно. Величина ***TP*** (True positive) содержит общее количество таких ответов.
- Объект имеет характеристику класса 0, а классификатор определяет характеристику как 1. Это означает, что классификатор сработал не верно. Величина ***FP*** (False positive) содержит общее количество таких ответов.
- Объект имеет характеристику класса 0 и классификатор определяет характеристику как 0. Это означает, что классификатор сработал верно. Величина ***TN*** (True negative) содержит общее количество таких ответов.
- Объект имеет характеристику класса 1, а классификатор определяет характеристику как 0. Это означает, что классификатор сработал не верно. Величина ***FN*** (False negative) содержит общее количество таких ответов.    

Идеальный классификатор должен все определять верно, поэтому первая метрика ***Accuracy*** определяет долю правильных ответов и считается следующим образом:
$$Accuracy=\frac{TP+TN}{TP+TN+FP+FN}$$
Однако эту метрику рекомендуется использовать только для сбалансированных классов. Простой пример: если данные содержат 50 положительных объектов и 950 отрицательных, а классификатор настроен выдавть всегда отрицательный ответ, то *Accuracy = 0.95*, т.е. 95% правильных ответов при абсолютно бесполезном классификаторе.  
Второй метрикой качества является точность - ***Precision***, которая считается следующим образом:
$$Precision=\frac{TP}{TP+FP}$$
*Precision* определяет долю положительных объектов, среди положительных ответов классификатора.  
Следующая метрика называется полнотой - ***Recall***, определяет долю выявленных классификатором положительных объектов, и рассчитывается так:
$$Recall=\frac{TP}{TP+FN}$$
На практике часто требуются хорошие значения и точности, и полноты, поэтому было разработано множество показателей на основе обеих метрик. Наиболее известная метрика - это F-мера:
$$F_\beta = (1 + \beta^2) \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{(\beta^2 \cdot \mathrm{precision}) + \mathrm{recall}} = \frac {(1 + \beta^2) \cdot \mathrm{TP} }{(1 + \beta^2) \cdot \mathrm{TP} + \beta^2 \cdot \mathrm{FN} + \mathrm{FP}}$$
F-мера - это среднее гармоническое точности и полноты. При плохих значениях одной из метрик (или обеих), F-мера устремляется к нулю. Параметр $\beta$ регулирует значимости метрик:
- Если $\beta>1$, то F-мера сильнее реагирует на долю ложноположительных ответов, т.е. на *Recall*;
- Если $\beta<1$, то F-мера чувствительнее к точности;
- Если $\beta=1$, то *Recall* и *Precision* равнозначны.
Часто используют именно метрику **$F_1$**:
$$F_1 = \frac{2\cdot\mathrm{precision} \cdot \mathrm{recall}}{ \mathrm{precision} + \mathrm{recall}}$$ 

Значительное отклонение метрик качества на обучающей и тестовой выборках может свидетельствовать о [переобучении модели](https://ru.wikipedia.org/wiki/Переобучение)

***scikit-learn***

Для разработки классификатора воспольземся библиотекой с инструментами машинного обучения [scikit-learn](https://scikit-learn.org/stable/) (sklearn).  
Нам потребуются 4 инструмента:
- ***classification_report*** из модуля [metrics](https://scikit-learn.org/stable/modules/classes.html?highlight=metrics#module-sklearn.metrics) сформирует метрики *Accuracy*, *Precision*, *Recall* и $F_1$;
- ***train_test_split*** из модуля [model_selection](https://scikit-learn.org/stable/modules/classes.html?highlight=sklearn.model_selection#module-sklearn.model_selection) поможет разбить выборку на обучающую и тестовую;
- ***StandardScaler*** из модуля [preprocessing](https://scikit-learn.org/stable/modules/classes.html?highlight=sklearn.preprocessing#module-sklearn.preprocessing) поможет преобразовать данные к стандартному виду;
- ***LogisticRegression*** из модуля [linear_model](https://scikit-learn.org/stable/modules/classes.html?highlight=sklearn.linear_model#module-sklearn.linear_model) поможет подобрать коэффициенты для логистической регрессии.

In [14]:
"""Импортируем необходимые инструменты"""

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
"""Разрабатываем классификатор"""

y = df.Office                                   # Определяем результирующий признак
X = df[['Order_time_sec', 'Delivery_time']]     # Определяем признаки

"""С помощью функции train_test_split разбиваем выборку на обучающую и тестовую. Параметр test_size указывает на 
   долю тестовой выборки. Если параметр random_state отличен от None, т.е. имеет конкретное значение, то 
   генерируемые псевдослучайные величины будут иметь одни и те же значения при каждом вызове, и мы будем каждый 
   раз получать одинаковое разбиение на обучающую и тестовую выборки. Условимся, что, если не указано другое, то
   по всему курсу обучения random_state=21"""
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

"""Функция StandardScaler приведет значения признаков к стандартному виду z = (x-u)/s, где u - среднее значение,
   а s - стандартное отклонение. Расчет среднего значения и стандартного отклонение осуществляется на выборке для 
   обучения, затем осуществляется преобразования данных для обучения и тестовых данных """
sc = StandardScaler()
sc.fit(X_train)                                  # Находим среднее и стандартное отклонение обучающей выборки
X_train_std = sc.transform(X_train)              # Преобразовываем обучающую выборку
X_test_std = sc.transform(X_test)                # Преобразовываем тестовую выборку

"""Формируем классификатор с помощью LogisticRegression. Коэффициенты уравнения логистической регрессии находятся
   на обучающей выборке, затем с помощью найденного уравнения предсказывается класс объекта на тестовой выборке"""
lr = LogisticRegression(random_state=21)
lr.fit(X_train_std, y_train)                     # Находим коэффициенты уравнения логистической регрессии
y_pred = lr.predict(X_test_std)                  # Определяем класс на значениях тестовых данных

print(classification_report(y_test, y_pred))     # Публикуем результаты качества модели на тестовой выборке

In [ ]:
y_pred

---

Для анализа полученного результата напишем функцию *Processing_results()*, которой передаются значения:
- sc - действующий (полученный при разработке классификатора) метод стандартизации данных;
- lr - классификатор;
- df - датафрейм;
- target - название столбца целевого признака;
- attribute_name - список названий столбцов признаков;
- name - строка названия анализируемого результата;
- add_schedule - список списков признаков, для которых необходимо построить дополнительные графики (необязательный параметр)

In [17]:
def Processing_results(sc, lr, df, target, attribute_name, name, add_schedule = []):
    X_std = sc.transform(df[attribute_name])
    new_col_name = []
    for i, col_name in enumerate(attribute_name):
        s = col_name + '_std_' + name
        df[s] = X_std[:,i]
        new_col_name.append(s)
    y_pred = 'y_pred_' + name
    Correct = 'Correct_' + name
    df[y_pred] = lr.predict(X_std)
    df[Correct] = df.Office == df[y_pred]
    print(classification_report(df.Office, df[y_pred]))
    sns.pairplot(df[[Correct] + new_col_name], hue = Correct, height=round(12/len(new_col_name), 1))
    plt.show()
    for k in range(len(add_schedule)):
        i,j = add_schedule[k]
        i = i+'_std_'+name
        j = j+'_std_'+name
        plt.scatter(df[i][True ^ df[Correct]], df[j][True ^ df[Correct]], color = 'red')
        plt.scatter(df[i][df[Correct]], df[j][df[Correct]])
        plt.legend(['Определены не верно', 'Определены верно'])
        plt.xlabel(i)
        plt.ylabel(j)
        plt.show()
    return df

In [ ]:
y = 'Ofice'
x = ['Order_time_sec', 'Delivery_time']
df = Processing_results(sc, lr, df, y, x, '1', [x])
df

In [ ]:
#df_test = pd.read_excel('iCafe_test.xlsx')
df_test = pd.read_excel('/content/drive/MyDrive/data/iCafe_test.xlsx')
df_test['Order_time_sec'] = pd.to_timedelta(df_test.Order_time)//np.timedelta64(1,'s')

y = 'Ofice'
x = ['Order_time_sec', 'Delivery_time']
df_test = Processing_results(sc, lr, df_test, y, x, '1', [x])
df_test

---

# Задание 1

Путем двойного контроля мы убедились, что модель не переобучена и демонстрирует не плохое качество, все метрики на общей обучающей выборке 0.88, на тестовой выборке - 0.91. Можно ли улучшить полученное решение? Для ответа на этот вопрос в начале еще раз посмотрим на графики:

In [ ]:
plt.subplot (2, 2, 1)
plt.scatter(df.Order_time_sec[df.Office == 1], df.Delivery_time[df.Office == 1], color = 'red')
plt.scatter(df.Order_time_sec[df.Office == 0], df.Delivery_time[df.Office == 0])
plt.legend(['В офис', 'На дом'])
plt.xlabel('Время поступления заказа (Order_time_sec)')
plt.ylabel('Время доставки заказа (Delivery_time)')
plt.plot([82000, 82000], [0,10], 'red')
plt.plot([82000, 86400], [10,10], 'red')
plt.title('Обучающая выборка, принадлежность к классам')

plt.subplot (2, 2, 2)
plt.scatter(df_test.Order_time_sec[df_test.Office == 1], df_test.Delivery_time[df_test.Office == 1], color = 'red')
plt.scatter(df_test.Order_time_sec[df_test.Office == 0], df_test.Delivery_time[df_test.Office == 0])
plt.legend(['В офис', 'На дом'])
plt.xlabel('Время поступления заказа (Order_time_sec)')
plt.ylabel('Время доставки заказа (Delivery_time)')
plt.plot([82000, 82000], [0,10], 'red')
plt.plot([82000, 86400], [10,10], 'red')
plt.title('Тестовая выборка, принадлежность к классам')

plt.subplot (2, 2, 3)
plt.scatter(df.Order_time_sec[True^df.Correct_1], df.Delivery_time[True^df.Correct_1], color = 'red')
plt.scatter(df.Order_time_sec[df.Correct_1], df.Delivery_time[df.Correct_1])
plt.legend(['False', 'True'])
plt.xlabel('Время поступления заказа (Order_time_sec)')
plt.ylabel('Время доставки заказа (Delivery_time)')
plt.plot([82000, 82000], [0,10], 'red')
plt.plot([82000, 86400], [10,10], 'red')
plt.title('Обучающая выборка, предсказание класса')

plt.subplot (2, 2, 4)
plt.scatter(df_test.Order_time_sec[True^df_test.Correct_1], df_test.Delivery_time[True^df_test.Correct_1], 
            color = 'red')
plt.scatter(df_test.Order_time_sec[df_test.Correct_1], df_test.Delivery_time[df_test.Correct_1])
plt.legend(['False', 'True'])
plt.xlabel('Время поступления заказа (Order_time_sec)')
plt.ylabel('Время доставки заказа (Delivery_time)')
plt.plot([82000, 82000], [0,10], 'red')
plt.plot([82000, 86400], [10,10], 'red')
plt.title('Тестовая выборка, предсказание класса')
plt.show()

На графиках мы видим одну и ту же область, где представители одного класса предсказываются неверно как на обучающей, так и на тестовой выборке. Попробуем исправить это!

1. Вам необходимо написать функцию time_change(df), которая записывает в датафрейм df столбец Order_time_sec_delta и возвращает полученный датафрейм. Столбец Order_time_sec_delta отличается от столбца Order_time_sec константой. В столбце Order_time_sec отметки времени расположены так, что последний заказ "на дом" (Office = 0) оказывается на отметке 86399 секунд от начала дня (т.е. ровно через сутки после начала). В этой функции нужно "сдвинуть" время всех заказов циклично влево, ближе к началу дня, на дельту, равную разнице \[24 часа - максимальное время заказа на дом во всем датафрейме].
2. Вам необходимо написать функцию log_reg(y, X), которая принимает на вход целевой признак y и столбцы датафрейма X, представляющие собой признаки. Функция возвращает обученные объекты sc и lr

In [ ]:
df[df.Office == 0].Order_time_sec.max()

In [22]:
"""Выполнение задания 1"""

def time_change(df):
  delta = 60*24*60 - df[df.Office == 0].Order_time_sec.max()
  df['Order_time_sec_delta'] = (df.Order_time_sec + delta - 1) % (24* 60 * 60)
  return df

def log_reg(y,X):
  y = df.Office                                         # Определяем результирующий признак
  X = df[['Order_time_sec_delta', 'Delivery_time']]     # Определяем признаки
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)
  sc = StandardScaler()
  sc.fit(X_train)                                       # Находим среднее и стандартное отклонение обучающей выборки
  X_train_std = sc.transform(X_train)                   # Преобразовываем обучающую выборку
  X_test_std = sc.transform(X_test)                     # Преобразовываем тестовую выборку
  lr = LogisticRegression(random_state=21)
  lr.fit(X_train_std, y_train)                          # Находим коэффициенты уравнения логистической регрессии
  y_pred = lr.predict(X_test_std)                       # Определяем класс на значениях тестовых данных
  return sc, lr
    

In [ ]:
"""Запустите, после написания функций"""

df = time_change(df)
y = 'Office'
x = ['Order_time_sec_delta', 'Delivery_time']
sc_delta, lr_delta = log_reg(df[y], df[x])
df = Processing_results(sc_delta, lr_delta, df, y, x, 'delta', [x])
df

In [ ]:
"""Запустите, если сработала предыдущая ячейка"""

df_test = time_change(df_test)
y = 'Office'
x = ['Order_time_sec_delta', 'Delivery_time']
df_test = Processing_results(sc_delta, lr_delta, df_test, y, x, 'delta', [x])
df_test

***Рекомендуется выполнить задание перед рассмотрением дальнейшего материала***

---

Мы улучшили качество классификатора за счет обеспечения монотонности целевой переменной от признаков, т.е. теперь выполняютя правила:
- чем больше момент времени поступления заказа, тем выше вероятность доставки на дом;
- чем выше время доставки, тем выше вероятность вероятность доставки на дом.

К сожалению, не всегда есть возможность обеспечить монотонность целевой переменной от признаков без применения каких-нибудь дополнительных инструментов.

Можно ли улучшить качество классификатора еще каким-нибудь способом? Это не только возможно, но и необходимо. 

Одним из способов улучшения классификатора является увеличение порядка пространства признаков, например, если на плоскости достаточно тяжело провести линию, разделяющую классы, то в трехмерном пространстве уже можно найти плоскость, которая сделает это лучше.

Все моменты времени необходимо преобразовывать через синус и косинус, формируя тем самым вместо одного два признака. Это становится тем более важным, если у нас вместо одного временного периода имеется несколько и существует возможность периодических колебаний.

Преобразования производятся по следующим формулам:
$$z_{sin} = sin\left(\frac{2\cdot\mathrm{\pi} \cdot \mathrm{x}}{ \mathrm{K} }\right)$$ 
$$z_{cos} = cos\left(\frac{2\cdot\mathrm{\pi} \cdot \mathrm{x}}{ \mathrm{K} }\right)$$ 

Здесь х - это значение момента времени, а К - значение временного периода в единицах измерения моментов времени.
Например: 
- если моменты времени измеряются в секундах, а временной период равен суткам, то К=86400;
- если моменты времени измеряются в днях, а временной период равен неделе, то К=7.

Проведем преобразование времени поступления заказа:

In [ ]:
df['Order_sin'] = np.sin(df.Order_time_sec*math.pi/43200)
df['Order_cos'] = np.cos(df.Order_time_sec*math.pi/43200)
df

Построим графики целевого показателя в пространстве полученных признаков:

In [ ]:
plt.scatter(df.Order_sin[df.Office == 1], df.Order_cos[df.Office == 1], color = 'red')
plt.scatter(df.Order_sin[df.Office == 0], df.Order_cos[df.Office == 0])
plt.legend(['В офис', 'На дом'])
plt.xlabel('Синус времени заказа')
plt.ylabel('Косинус времени заказа')
plt.show()

In [ ]:
sns.scatterplot(x = 'Order_sin', y = 'Order_cos', data = df, hue = 'Office')
plt.show()

In [ ]:
sns.pairplot(df[['Office', 'Delivery_time', 'Order_sin', 'Order_cos']], hue="Office", height=4)
plt.show()

Построим классификатор на преобразованых данных

In [ ]:
y = df.Office
X = df[['Order_sin', 'Order_cos', 'Delivery_time']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

sc_2 = StandardScaler()
sc_2.fit(X_train)
X_train_std = sc_2.transform(X_train)
X_test_std = sc_2.transform(X_test)

lr_2 = LogisticRegression(random_state=21)
lr_2.fit(X_train_std, y_train)

y_pred_2 = lr_2.predict(X_test_std)

print(classification_report(y_test, y_pred_2))

y = 'Office'
X = ['Order_sin', 'Order_cos', 'Delivery_time']
x = ['Order_sin', 'Order_cos']
df = Processing_results(sc_2, lr_2, df, y, X, '2', [x])
df

In [ ]:
df_test['Order_sin'] = np.sin(df_test.Order_time_sec*math.pi/43200)
df_test['Order_cos'] = np.cos(df_test.Order_time_sec*math.pi/43200)

y = 'Office'
X = ['Order_sin', 'Order_cos', 'Delivery_time']
x = ['Order_sin', 'Order_cos']
df_test = Processing_results(sc_2, lr_2, df_test, y, X, '2', [x])
df_test

На общей выборке метрики показывают такой же результат, как и в задании 1, на тестовой выборе результат немного хуже.

---

# Задание 2

Возьмите данные со смещенным временем из задания 2. Произведите преобразование через синус и косинус. Постройте классификатор. Сравните результаты.

In [ ]:
"""Решение задания 2"""
df['Order_sin_delta'] = np.sin(df.Order_time_sec_delta*math.pi/43200)
df['Order_cos_delta'] = np.cos(df.Order_time_sec_delta*math.pi/43200)
df

In [ ]:
plt.scatter(df.Order_sin_delta[df.Office == 1], df.Order_cos_delta[df.Office == 1], color = 'red')
plt.scatter(df.Order_sin_delta[df.Office == 0], df.Order_cos_delta[df.Office == 0])
plt.legend(['В офис', 'На дом'])
plt.xlabel('Синус времени заказа')
plt.ylabel('Косинус времени заказа')
plt.show()

In [ ]:
sns.scatterplot(x = 'Order_sin_delta', y = 'Order_cos_delta', data = df, hue = 'Office')
plt.show()

In [ ]:
sns.pairplot(df[['Office', 'Delivery_time', 'Order_sin_delta', 'Order_cos_delta']], hue="Office", height=4)
plt.show()

In [ ]:
y = df.Office
X = df[['Order_sin_delta', 'Order_cos_delta', 'Delivery_time']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

sc_2 = StandardScaler()
sc_2.fit(X_train)
X_train_std = sc_2.transform(X_train)
X_test_std = sc_2.transform(X_test)

lr_2 = LogisticRegression(random_state=21)
lr_2.fit(X_train_std, y_train)

y_pred_2 = lr_2.predict(X_test_std)

print(classification_report(y_test, y_pred_2))

y = 'Office'
X = ['Order_sin_delta', 'Order_cos_delta', 'Delivery_time']
x = ['Order_sin_delta', 'Order_cos_delta']
df = Processing_results(sc_2, lr_2, df, y, X, '2', [x])
df

In [ ]:
df_test['Order_sin_delta'] = np.sin(df_test.Order_time_sec_delta*math.pi/43200)
df_test['Order_cos_delta'] = np.cos(df_test.Order_time_sec_delta*math.pi/43200)

y = 'Office'
X = ['Order_sin_delta', 'Order_cos_delta', 'Delivery_time']
x = ['Order_sin_delta', 'Order_cos_delta']
df_test = Processing_results(sc_2, lr_2, df_test, y, X, '2', [x])
df_test